<a href="https://colab.research.google.com/github/baltuna/LT4DH/blob/main/LT4DH_ELTeC_NER_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting part of speech from ELTeC-ENG

Adaptation of a great Colab by Borja Navarro for the LT4DH course in the University of the Basque Country.

This version (to be cleaned) uses English resources in contrast to the Spanish one used by Borja Navarro.

Original reference here:

Borja Navarro Colorado | University of Alicante

In this case, the information about part of speech has not been manually annotated in the corpus. It is necessary first analyze the novels with a NLP system and then extract the linguistic information. The NLP system used is [SpaCy](https://spacy.io/).

The notebook shows:

- how to open a novel from ELTeC in COLAB,
- how to activate SpaCy in COLAB,
- how to analyze the novel with SpaCy, and
- how to extract information about Part of Speec.


## Loading ELTeC-ENG corpus in Colab

In [ ]:
import zipfile

!wget "https://github.com/COST-ELTeC/ELTeC-eng/archive/refs/heads/master.zip" # paste here corpus url

zip_ref = zipfile.ZipFile('master.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts files here (/content/)
zip_ref.close() 
!rm master.zip #Removes ZIP to save space

--2023-03-23 16:17:12--  https://github.com/COST-ELTeC/ELTeC-eng/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/COST-ELTeC/ELTeC-eng/zip/refs/heads/master [following]
--2023-03-23 16:17:12--  https://codeload.github.com/COST-ELTeC/ELTeC-eng/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [     <=>            ]  91.32M  4.11MB/s    in 22s     

2023-03-23 16:17:35 (4.14 MB/s) - ‘master.zip’ saved [95756685]



## SpaCy: download and installing

[SpaCy](https://spacy.io/) is a NLP system. It analyzes part of speech and lemmas, sintax (dependencies) and named entities. 

Three steps:

1. Import SpaCy to Colab
2. Download language module (English)
3. Activate module


In [ ]:
import spacy

!python -m spacy download en_core_web_sm #Download Spanish module (the "small" module in this case: "sm").

import en_core_web_sm
nlp_eng = en_core_web_sm.load() #Load Spanish analyzer in "nlp_esp".

## Analyzing a novel from ELTeC-ENG

Once we have downloaded the corpus and activated SpaCy, let's analyze one novel.

First, select from the corpus [ELTeC-ENG](https://github.com/COST-ELTeC/ELTeC-eng/tree/master/level1) a novel and copy the file name. Then paste the name in the variable "novela_name". In this example, we will analyze the novel [*Yeast*](https://github.com/COST-ELTeC/ELTeC-eng/tree/master/level1/ENG18510_Kingsley.xml): ENG18510_Kingsley.xml

In [ ]:
import os
from bs4 import BeautifulSoup

novela_name = "ENG19201_Arlen.xml" # Put here the name of the file
dir_in = "/content/ELTeC-eng-master/level1/"

novela_text = '' 

print('Analyzing', novela_name)

ficheroEntrada = dir_in + novela_name
with open(ficheroEntrada, 'r') as tei: #Opens the file
  print("Opening the file and extracting text")
  soup = BeautifulSoup(tei, 'xml') #Parse the XML
  capitulos = soup.find_all(type="chapter") #Only chapters are taking into account. No letters (To Do)
  for cap in capitulos:
    parrafos = cap.find_all('p') #Extract all paragraphs of each chapter
    for parrafo in parrafos:
      #print(parrafo.text)
      novela_text+=parrafo.text+'\n'

#print('Analyzing PoS and lemmas')
analisis = nlp_eng(novela_text) #Here the novel is analyzed with SpaCy. All the analysis is stored in "analisis" variable.
print('Done!')

Now all the analysis is stored in "analisis" variable. It only remains to iterate over the variable and extract the information: in this case, named entities. How to extract information about syntax, named entities, etc. see [SpaCy 101](https://spacy.io/usage/spacy-101)

In [ ]:
from google.colab import files

entitateak = []

def show_ents(doc): #funzionatzen du
  if doc.ents: 
    for ent in doc.ents: 
     # print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_) 
      print(ent.text+'\t'+str(ent.start_char) +'\t'+ str(ent.end_char) +'\t'+ent.label_) 
    else: print(str(len(doc.ents))+ ' were found')

def one_ent(doc): #funzionatzen du
  if doc.ents: 
    for ent in doc.ents: 
      if ent.label_ == 'LOC':
        entitateak.append(ent)
        print(ent.text+'\t'+str(ent.start_char) +'\t'+ str(ent.end_char) +'\t'+ent.label_)
    else: print(str(len(entitateak))+ ' entities were found')

def two_ents(doc): #funzionatzen du
  if doc.ents: 
    for ent in doc.ents: 
      if ent.label_ == 'LOC' or ent.label_ == 'GPE':
        entitateak.append(ent)
        print(ent.text+'\t'+str(ent.start_char) +'\t'+ str(ent.end_char) +'\t'+ent.label_)
    else: print(str(len(entitateak))+ ' location entities were found')



#################################PROGRAMA#########################################

#show_ents(analisis)

#one_ent(analisis)

two_ents(analisis)

########################ENTITY TYPES#############################
# CARDINAL, PERSON, ORG, GPE, NORP, EVENT, DATE, WORK_OF_ART, TIME, PRODUCT, LOC LANGUAGE, FAC, ORDINAL


####################################EXTRA###########################################

locs = []
unique_loc = []

with open("arlen.csv", "w") as output:
  for ent in analisis.ents: 
    if ent.label_ == 'LOC' or ent.label_ == 'GPE':
      locs.append(ent.lemma_) #
      if ent.lemma_ not in unique_loc:
        unique_loc.append(ent.lemma_)
        output.write(f"{ent.text}\n")
    # elif token.pos_ == 'VERB':
    #   verbs.append(token.lemma_) #
    #   if token.lemma_ not in unique_verb:
    #      unique_verb.append(token.lemma_)
    # elif token.pos_ == 'ADJ':
    #   adjs.append(token.lemma_) #
    #   if token.lemma_ not in unique_adj:
    #      unique_adj.append(token.lemma_)





# entitateak_out = ''
# if doc.ents:
#   for entitate in entitateak:
#    entitateak.sort()
#    entity = entitateak[0]
#    enttype = entitateak[3]
#    entitateak_out+=str(entity)+'\t'+str(enttype)+'\n'

# print(entitateak_out)


#out = open('chosen_entities.csv', 'w') #Opens a file in write mode ("w").
#out.write(entitateak) # "Writes" the content of authors_titles_out in the file
#out.close() #Closes the file
#files.download('chosen_entities.csv')


##################################################################################




In [ ]:
#num_pers = str(len(pers))

#print(num_pers)

#NVA = '\tNovel\tORG\tPER\tGEO\tGPE\tTIM\tART\tEVE\tNAT\n' #
#NVA = '\tNovel\tNouns\tVerbs\tAdjectives\tUnique_nouns\tUnique_verbs\tUnique_adjs\tNER\n' #

#nom_novela = 'Yeats'
#nom_novela = novela_name

#novela=novela_text

#unique_ent=[]

#orgs=[]
#pers=[]
#geos=[]
#geps=[]
#tims=[]
#arts=[]
#eves=[]
#nats=[]





#for ent in analisis.ents: 
 # if ent.ent_type == 'B-geo':
  #  eves.append(token.lemma_) #
 # elif token.pos_ == 'VERB':
  #  verbs.append(token.lemma_) #
   # if token.lemma not in unique_verb:
    #   unique_verb.append(token.lemma)
 # elif token.pos_ == 'ADJ':
  #  adjs.append(token.lemma_) #
   # if token.lemma not in unique_adj:
    #   unique_adj.append(token.lemma)


#num_nouns = str(len(nouns))
#num_verbs = str(len(verbs))
#num_adjs = str(len(adjs))
#num_uninouns = str (len(unique_noun))
#num_univerbs = str (len(unique_verb))
#num_uniadjs = str (len(unique_adj))
#num_eves = str(len(ners))

#NVA += '\t'+nom_novela+'\t'+num_nouns+'\t'+num_verbs+'\t'+num_adjs+'\t'+num_uninouns+'\t'+num_univerbs+'\t'+num_uniadjs+'\t'+num_eves+'\n' #

#print(NVA)

#salida = open('analisis_soloNombres.txt', 'w') #Crea fichero, etc.
#salida.write(corpus_soloNombres)
#salida.close()
#files.download('analisis_soloNombres.txt')

#out = open('author_titles.csv', 'w') #Opens a file in write mode ("w").
#out.write(authors_titles_out) # "Writes" the content of authors_titles_out in the file
#out.close() #Closes the file
#files.download('author_titles.csv')